In [ ]:
import os
import sounddevice as sd
import numpy as np
from datetime import datetime
import time
# Create the "Recordings" folder if it doesn't exist
folder_name = "Recordings"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Set the sample rate and duration for recording
sample_rate = 44100  # CD quality
duration = 5  # Duration of each recording in seconds

# Initialize recording parameters
frames = []
start_time = None
def calculate_threshold(indata):
    rms = np.sqrt(np.mean(np.square(indata)))
    threshold = rms * 100
    return threshold

def start_recording(indata, data_frames, ctime, status):
    global start_time, frames
    if status:
        print("Error:", status)
    frames.append(indata.copy())
    print(np.max(indata) )
    if start_time is None:
        threshold = calculate_threshold(indata)
        start_time = time.time()
        print("start")
        
    else:
        # Check for silence based on the calculated threshold
        if np.max(indata) < threshold:
            
            # Stop recording and save the recorded audio as a WAV file
            filename = f"{start_time.strftime('%Y-%m-%d_%H-%M-%S')}.wav"
            filepath = os.path.join(folder_name, filename)
            sd.write(filepath, np.concatenate(frames), sample_rate)
            print(f"Saved recording: {filename}")
            frames.clear()
            start_time = None

# Start the audio stream
with sd.InputStream(callback=start_recording, channels=1, samplerate=sample_rate,device=6):
    print("Listening for speech...")
    while True:
        try:
            sd.sleep(duration * 1000)
        except KeyboardInterrupt:
            if start_time is not None:
                # Stop recording if interrupted by user
                print("Recording stopped by user.")
                start_recording(None, None, None, None)
            break
